In [1]:
import numpy as np
import pandas as pd
import sklearn

In [2]:
import sys
!{sys.executable} -m pip install xgboost


In [3]:
df=pd.read_csv("aqardata_2.csv")

In [4]:


df.info()
df.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2951 entries, 0 to 2950
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mainlocation  2951 non-null   object 
 1   sublocation   2276 non-null   object 
 2   neighborhood  2951 non-null   object 
 3   frontage      2951 non-null   object 
 4   purpose       2604 non-null   object 
 5   streetwidth   2948 non-null   float64
 6   size          2951 non-null   int64  
 7   Pricepm       2951 non-null   int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 184.6+ KB


,mainlocation,sublocation,neighborhood,frontage,purpose,streetwidth,size,Pricepm
0,الرياض,غرب الرياض,حي ظهرة لبن,شمال,سكني,20.0,727,1800
1,بريدة,NaN,حي مشعل,غرب,NaN,15.0,450,950
2,الخبر,NaN,حي الحمرا,غرب,تجاري,100.0,1450,3500
3,الخبر,NaN,حي الحزام الاخضر,شرق,NaN,15.0,440,2700
4,بريدة,NaN,حي الرحاب,جنوب غربي,NaN,40.0,784,950


In [5]:
df.drop(labels="sublocation",axis=1,inplace=True)
df['streetwidth'] = df['streetwidth'].fillna(int(np.mean(df["streetwidth"])))
df['purpose'].fillna('سكني', inplace = True)
df = pd.get_dummies(data=df, columns=['mainlocation', 'purpose', 'frontage', 'neighborhood'])


In [16]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2951 entries, 0 to 2950
Columns: 350 entries, streetwidth to neighborhood_حي هجر
dtypes: float64(1), int64(2), uint8(347)
memory usage: 1.0 MB


In [7]:
df.head()

,streetwidth,size,Pricepm,mainlocation_ابها,mainlocation_ابو عريش,mainlocation_احد رفيده,mainlocation_الجموم,mainlocation_الخبر,mainlocation_الخرج,mainlocation_الدرعية,...,neighborhood_حي مذينب,neighborhood_حي مرخ,neighborhood_حي مركز الملك عبدالله للدراسات والبحوث,neighborhood_حي مريخ,neighborhood_حي مشرف,neighborhood_حي مشرفة,neighborhood_حي مشعل,neighborhood_حي منطقة المرافق,neighborhood_حي نمار,neighborhood_حي هجر
0,20.0,727,1800,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,15.0,450,950,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,100.0,1450,3500,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15.0,440,2700,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,40.0,784,950,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X= df.drop('Pricepm', axis = 1)
X

,streetwidth,size,mainlocation_ابها,mainlocation_ابو عريش,mainlocation_احد رفيده,mainlocation_الجموم,mainlocation_الخبر,mainlocation_الخرج,mainlocation_الدرعية,mainlocation_الدلم,...,neighborhood_حي مذينب,neighborhood_حي مرخ,neighborhood_حي مركز الملك عبدالله للدراسات والبحوث,neighborhood_حي مريخ,neighborhood_حي مشرف,neighborhood_حي مشرفة,neighborhood_حي مشعل,neighborhood_حي منطقة المرافق,neighborhood_حي نمار,neighborhood_حي هجر
0,20.0,727,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,15.0,450,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,100.0,1450,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15.0,440,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,40.0,784,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2946,20.0,354,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2947,100.0,143875,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2948,15.0,934,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2949,15.0,832,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
y=df["Pricepm"]
y

0       1800
1        950
2       3500
3       2700
4        950
        ... 
2946    1200
2947     125
2948     550
2949    2900
2950     166
Name: Pricepm, Length: 2951, dtype: int64

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [30]:
import xgboost as xgb
gbr=xgb.XGBRegressor(n_estimators = 750,
                         max_depth = 15,
                         verbosity=0)

gbr.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=15, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=750, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [31]:
y_pred = gbr.predict(X_test)

In [32]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
print("Mean absolute error:     %.2f" % mean_absolute_error(y_test, y_pred))
print("root Mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred)))

Mean absolute error:     397.83
root Mean squared error: 718.31


In [22]:

print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))


Mean squared error: 515966.85
